In [ ]:
!pip3 install tensorflow_text>=2.0.0

In [ ]:
#!pip3 install tensorflow_text>=2.0.0
import tensorflow_hub as hub
import numpy as np
import tensorflow_text

x_train = open('/content/drive/MyDrive/Colab Notebooks/film_plots_lemma.txt',
               encoding='utf-8').read().split('\n')

y_train = open('/content/drive/MyDrive/Colab Notebooks/wiki_titles.txt',
               encoding='utf-8').read().split('\n')

x_test = open('/content/drive/MyDrive/Colab Notebooks/test_data_lemma.txt',
              encoding='utf-8').read().split('\n')

y_test = open('/content/drive/MyDrive/Colab Notebooks/test_titles.txt',
              encoding='utf-8').read().split('\n')

x_summary_test = open('/content/drive/MyDrive/Colab Notebooks/summary_lemma.txt',
                      encoding='utf-8').read().split('\n')

y_summary_test = open('/content/drive/MyDrive/Colab Notebooks/summary_titles.txt',
                      encoding='utf-8').read().split('\n')

print(len(x_train) == len(y_train))
print(len(x_test) == len(y_test))
print(len(x_summary_test) == len(y_summary_test))

model = hub.load('https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3')

True
True
True


In [ ]:
import pandas as pd


def ap(relev, k):    # average presicion
    ap = []
    for i in range(1, k):
        if relev[i-1] is 1:    # if doc is relevant
            ap.append(sum(relev[:i])/i)    # summary of precisions
    try: 
        ap = sum(ap)/sum(relev)
    except ZeroDivisionError:
        ap = 0.0
    return ap


def evaluation(query, relev, index=0, k=5):    # mean average precision (5)
    prec = round(sum(relev)/k, 4)    # rank is not take into account
    k += 1
    avp = ap(relev, k)    # rank is take into account
    evaluat = pd.DataFrame({'precision': prec,
                            'average_precision': avp},
                            index=[index])
    return evaluat


def calc_map(query, film, predictions, k, index):
    relev = [0] * k
    for i, pred in enumerate(predictions):
        if y_train[pred] == film:
            relev[i] = 1
    if relev != [0] * k:   
        df = evaluation(query, relev, index, k)
    else:
        df = pd.DataFrame({'precision': 0.0,
                           'average_precision': 0.0},
                           index=[index])
    return df


# map = sum(ap)/Q    # Q - number of quaries
# recall = sum(p)/Q

In [ ]:
!pip install annoy

In [19]:
from annoy import AnnoyIndex # https://github.com/spotify/annoy
import numpy as np
import time

start = time.time()

f = 512
t = AnnoyIndex(f, 'angular')  # Length of item vector that will be indexed
for idx, text in enumerate(x_train):
    v = model(text)[0]
    try:
        t.add_item(idx, v)
    except ValueError:
        continue

t.build(100) # 100 trees
t.save('test.ann')

end = time.time()
print(end - start)

5598.728063821793


In [20]:
u = AnnoyIndex(f, 'angular')
u.load('test.ann') # super fast, just map the file

df = pd.DataFrame(columns=['precision', 'average_precision'])

for idx, (query, film) in enumerate(zip(x_test, y_test)):
    v = model(query)[0]
    top_20 = u.get_nns_by_vector(v, 20) # find the 20 nearest neighbors

    df2 = calc_map(query, film, top_20, 20, idx)
    df = df.append(df2)

In [21]:
MAP = pd.DataFrame(columns=['recall', 'MAP'])
df2 = pd.DataFrame({'recall': len([1 for prec in df.precision if prec != 0])/len(x_test),
                    'MAP': sum(df.average_precision)/len(x_test)},
                    index=['vk'])
MAP = MAP.append(df2)
MAP

,recall,MAP
vk,0.009,0.003011


In [22]:
df = pd.DataFrame(columns=['precision', 'average_precision'])

for idx, (query, film) in enumerate(zip(x_summary_test, y_summary_test)):
    v = model(query)[0]
    top_20 = u.get_nns_by_vector(v, 20) # find the 20 nearest neighbors

    df2 = calc_map(query, film, top_20, 20, idx)
    df = df.append(df2)

In [23]:
df2 = pd.DataFrame({'recall': len([1 for prec in df.precision if prec != 0])/len(x_summary_test),
                    'MAP': sum(df.average_precision)/len(x_summary_test)},
                    index=['wiki summaries'])
MAP = MAP.append(df2)
MAP

,recall,MAP
vk,0.009,0.003011
wiki summaries,0.045,0.021145


In [25]:
x_0 = model(x_test[28])  # Дом у озера
x_1 = model(x_test[462])  # Дом у озера
x_2 = model(x_test[2])  # neg

y = model(x_train[9469])  # Дом у озера

x_0_sim = np.inner(x_0, y)[0][0]
x_1_sim = np.inner(x_1, y)[0][0]
x_2_sim = np.inner(x_2, y)[0][0]

print(f'Similarity for two matching items: {x_0_sim:.2f} and {x_1_sim:.2f}')
print( f'Similarity for negative item: {x_2_sim:.2f}')

Similarity for two matching items: 0.35 and 0.19
Similarity for negative item: 0.06
